# EDA

# 1. Data

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
# 경로 설정
data_path = '../data'

### 데이터 불러오기
- history_data: 시청 시작 데이터
- watch_e_data: 시청 종료 데이터
- buy_data: 구매 이력 데이터
- search_data: 검색을 통한 시청 데이터
- meta_data: 콘텐츠 일반 메타 데이터
- meta_data_plus: 콘텐츠 확장 정보
- profile_data: 프로필 정보

In [3]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
watch_e_df = pd.read_csv(os.path.join(data_path, 'watch_e_data.csv'), encoding='utf-8')
buy_df = pd.read_csv(os.path.join(data_path, 'buy_data.csv'), encoding='utf-8')
search_df = pd.read_csv(os.path.join(data_path, 'search_data.csv'), encoding='utf-8')
meta_plus_df = pd.read_csv(os.path.join(data_path, 'meta_data_plus.csv'), encoding='utf-8')

/var/folders/5k/c00qxxss767flx33ndntcx3m0000gn/T/ipykernel_17161/1515175036.py:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')


In [4]:
print('history shape:', history_df.shape)
print('profile shape:', profile_df.shape)
print('meta shape:', meta_df.shape)
print('watch e shape:', watch_e_df.shape)
print('buy shape:', buy_df.shape)
print('search shape:', search_df.shape)
print('meta plus shape:', meta_plus_df.shape)

history shape:  (1005651, 8)
profile shape:  (8311, 9)
meta shape:  (42602, 16)
watch e shape:  (892794, 8)
buy shape:  (150, 6)
search shape:  (6474, 5)
meta plus shape:  (767948, 4)


# 2. history

In [5]:
### 시청 시작 데이터 ###
# profile_id, album_id, payment, continuous_play 사용할 듯
# payment와 continuous_play는 어떻게 사용할지 고민해보기
# -> continuous_play(연속 재생 여부)는 N이라면 자동으로 재생된 것이 아니기 때문에 더 높은 가치를 가질 수도 있다고 생각
history_df

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,Y,N
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,Y,N
1005648,33032,20220427155839,20220427155810,MKID003,125,NaN,Y,N
1005649,33032,20220427155706,20220427155838,MKID003,125,NaN,Y,N


In [6]:
# payment만 null 값 존재
history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005651 entries, 0 to 1005650
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   profile_id       1005651 non-null  int64  
 1   ss_id            1005651 non-null  int64  
 2   log_time         1005651 non-null  int64  
 3   act_target_dtl   1005651 non-null  object 
 4   album_id         1005651 non-null  int64  
 5   payment          73486 non-null    float64
 6   continuous_play  1005651 non-null  object 
 7   short_trailer    1005651 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 61.4+ MB


In [7]:
history_df.isna().sum()/len(history_df)

profile_id         0.000000
ss_id              0.000000
log_time           0.000000
act_target_dtl     0.000000
album_id           0.000000
payment            0.926927
continuous_play    0.000000
short_trailer      0.000000
dtype: float64

In [8]:
print('사용자 수:', history_df['profile_id'].nunique())
print('album 수:', history_df['album_id'].nunique())

사용자 수: 8311
album 수: 20695


# 3. profile

In [9]:
### 프로필 정보 ###
profile_df

,profile_id,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,3,F,5,P02,P04,P07,K01,K03,K04
1,5,M,5,P07,P08,P06,K05,K08,K09
2,7,F,9,P05,P03,NaN,K06,K04,NaN
3,12,M,6,P03,P06,P02,K09,K07,K03
4,16,F,12,P03,P06,P01,K01,K06,K04
...,...,...,...,...,...,...,...,...,...
8306,33022,M,1,P04,NaN,NaN,K04,K08,NaN
8307,33023,M,5,P06,P03,P07,K08,K04,K05
8308,33026,F,8,P01,P03,P08,K05,K09,K06
8309,33027,F,4,P04,P05,P06,K03,K01,K05


In [10]:
# 부모 관심 키워드 2,3과 아이 관심 키워드 2,3은 null값이 존재하지만 1순위는 결측치 없음
profile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8311 entries, 0 to 8310
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   profile_id                8311 non-null   int64 
 1   sex                       8311 non-null   object
 2   age                       8311 non-null   int64 
 3   pr_interest_keyword_cd_1  8311 non-null   object
 4   pr_interest_keyword_cd_2  6778 non-null   object
 5   pr_interest_keyword_cd_3  6231 non-null   object
 6   ch_interest_keyword_cd_1  8311 non-null   object
 7   ch_interest_keyword_cd_2  6618 non-null   object
 8   ch_interest_keyword_cd_3  6029 non-null   object
dtypes: int64(2), object(7)
memory usage: 584.5+ KB


In [11]:
profile_df.isna().sum()/len(profile_df)

profile_id                  0.000000
sex                         0.000000
age                         0.000000
pr_interest_keyword_cd_1    0.000000
pr_interest_keyword_cd_2    0.184454
pr_interest_keyword_cd_3    0.250271
ch_interest_keyword_cd_1    0.000000
ch_interest_keyword_cd_2    0.203706
ch_interest_keyword_cd_3    0.274576
dtype: float64

# 4. meta

In [12]:
### 콘텐츠 일반 메타 데이터 ###
meta_df

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7
0,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
1,750,우리는 친구,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
2,2131,타요의 첫 운행,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
3,2625,길 잃은 타요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
4,2594,새내기 꼬마 버스의 하루,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42597,39873,로티프렌즈와 색칠놀이! - 그리피 ＆ 사탕 바구니,로티프렌즈 미술놀이,키즈,놀이교실,NaN,한국,477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42598,39874,로티프렌즈와 색칠놀이! - 베블리 ＆ 꽃,로티프렌즈 미술놀이,키즈,놀이교실,NaN,한국,466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42599,4779,손가락을 빨게 돼요,4-5세,키즈,책,NaN,한국,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42600,4779,손가락을 빨게 돼요,아이맘콕,키즈,책,NaN,한국,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# run_time(콘텐츠 길이)은 watch_e_data의 watch_time(실제 시청 시간)과 함께 사용할 수 있을 듯
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42602 entries, 0 to 42601
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   album_id     42602 non-null  int64  
 1   title        42602 non-null  object 
 2   sub_title    42602 non-null  object 
 3   genre_large  42602 non-null  object 
 4   genre_mid    42602 non-null  object 
 5   genre_small  13419 non-null  object 
 6   country      33734 non-null  object 
 7   run_time     42602 non-null  int64  
 8   onair_date   5344 non-null   float64
 9   cast_1       27603 non-null  object 
 10  cast_2       22048 non-null  object 
 11  cast_3       16463 non-null  object 
 12  cast_4       12485 non-null  object 
 13  cast_5       6382 non-null   object 
 14  cast_6       2609 non-null   object 
 15  cast_7       762 non-null    object 
dtypes: float64(1), int64(2), object(13)
memory usage: 5.2+ MB


In [14]:
meta_df.isna().sum()/len(meta_df)

album_id       0.000000
title          0.000000
sub_title      0.000000
genre_large    0.000000
genre_mid      0.000000
genre_small    0.685015
country        0.208159
run_time       0.000000
onair_date     0.874560
cast_1         0.352073
cast_2         0.482466
cast_3         0.613563
cast_4         0.706939
cast_5         0.850195
cast_6         0.938759
cast_7         0.982114
dtype: float64

In [15]:
print('고유 앨범 수:', meta_df['album_id'].nunique())

고유 앨범 수: 39875


In [16]:
print('고유 title 수:', meta_df['title'].nunique())

고유 title 수: 36185


In [17]:
print('meta_data 총 데이터 개수:', meta_df.shape[0])

meta_data 총 데이터 개수: 42602


In [18]:
titles = defaultdict(set)
for idx, row in meta_df.iterrows():
    titles[row['title']].add(row['album_id'])

In [27]:
titles['우리는 친구']

{416, 750, 11266, 13478, 25220, 33497}

In [28]:
# album_id와 title이 같다면 sub_title이 달라도 나머지 정보가 다 같음 -> sub_title은 삭제해도 될 듯!
meta_df[meta_df['title'] == '우리는 친구']

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7
1,750,우리는 친구,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
8266,25220,우리는 친구,깨비키즈 똘이야 학교가자,키즈,놀이교실,생활교육,NaN,227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17153,416,우리는 친구,NEW 뽀롱뽀롱 뽀로로1,키즈,TV만화,NaN,한국,305,NaN,뽀로로,크롱,루피,에디,NaN,NaN,NaN
17154,416,우리는 친구,만 3세를 위한 영상,키즈,TV만화,NaN,한국,305,NaN,뽀로로,크롱,루피,에디,NaN,NaN,NaN
17155,416,우리는 친구,체험동화 미리보기,키즈,TV만화,NaN,한국,305,NaN,뽀로로,크롱,루피,에디,NaN,NaN,NaN
17476,33497,우리는 친구,랄랄라 뿌우,키즈,창의학습,NaN,한국,749,NaN,뿌우,NaN,NaN,NaN,NaN,NaN,NaN
28364,13478,우리는 친구,뽀뽀뽀 친구친구,키즈,예능,NaN,한국,1586,NaN,뽀미언니,알루,점핑맨,느리,토리,우리,NaN
34566,11266,우리는 친구,띠띠뽀의 스토리＆송,키즈,TV만화,NaN,한국,1437,NaN,띠띠뽀와 친구들,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
meta_df = meta_df.drop(['sub_title'], axis = 1).drop_duplicates().reset_index(drop = True)
meta_df.shape

(39875, 15)

# 5. watch_e

In [30]:
### 시청 종료 데이터 ###
# profile_id, album_id, watch_time
# continuous_play(연속 재생 여부)-> 쓸지 말지 고민해보기
watch_e_df

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
0,3,20220301115653,20220301115805,MKID049,15,46,46,1
1,3,20220301115653,20220301115954,MKID049,16,104,105,1
2,3,20220301115653,20220301120114,MKID049,17,76,76,1
3,3,20220301115653,20220301120226,MKID049,18,67,68,1
4,3,20220301115653,20220301120400,MKID049,19,90,90,0
...,...,...,...,...,...,...,...,...
892789,33032,20220427155091,20220427155653,MKID049,381,463,464,1
892790,33032,20220427155091,20220427155694,MKID049,381,462,463,1
892791,33032,20220427155839,20220427155826,MKID049,125,10,520,0
892792,33032,20220427155706,20220427155836,MKID049,125,6,521,0


In [31]:
# 결측치 없음
watch_e_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892794 entries, 0 to 892793
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   profile_id       892794 non-null  int64 
 1   ss_id            892794 non-null  int64 
 2   log_time         892794 non-null  int64 
 3   act_target_dtl   892794 non-null  object
 4   album_id         892794 non-null  int64 
 5   watch_time       892794 non-null  int64 
 6   total_time       892794 non-null  int64 
 7   continuous_play  892794 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 54.5+ MB


In [32]:
# watch_e_data의 total_time과 meta_data의 run_time 모두 콘텐츠 길이를 나타내므로 같은 값을 가지는지 확인
watch_temp = watch_e_df[['profile_id', 'album_id', 'watch_time', 'total_time']]
meta_time = meta_df[['album_id', 'run_time']]
watch_time = pd.merge(watch_temp, meta_time, on='album_id', how='left')
watch_time

,profile_id,album_id,watch_time,total_time,run_time
0,3,15,46,46,46
1,3,16,104,105,105
2,3,17,76,76,76
3,3,18,67,68,68
4,3,19,90,90,90
...,...,...,...,...,...
892789,33032,381,463,464,462
892790,33032,381,462,463,462
892791,33032,125,10,520,520
892792,33032,125,6,521,520


In [33]:
count = 0
for idx, row in watch_time.iterrows():
    if abs(row['total_time'] - row['run_time']) > 60:
        count += 1

In [34]:
count # watch_e_data의 total_time(콘텐츠 길이)과 meta_data의 run_time(콘텐츠 길이)이 60초 이상 차이나는 것의 개수

28282

In [35]:
# 콘텐츠 전체 길이와 시청 시간의 비율을 사용해서 추천 성능을 더 높이고 싶은데
# total_time과 run_time이 많이 차이나는 것을 어떻게 처리해야 할지 모르겠음..
# 과연 둘 중에 더 정확한 정보는 무엇인가...?

# 6. buy

In [36]:
### 구매 이력 데이터 ###
# 데이터 매우 적음
# 어떻게 활용할 수 있을지 추후 고민
buy_df

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment
0,255,20220426153942,20220426154128,MKID004,3767,500
1,1603,20220309163919,20220309164043,MKID004,1640,300
2,5303,20220411080324,20220411082725,MKID004,3821,500
3,5303,20220411080324,20220411084013,MKID004,3822,500
4,5303,20220411080324,20220411085607,MKID004,3823,500
...,...,...,...,...,...,...
145,23882,20220421182514,20220421185231,MKID004,7441,300
146,23882,20220421182514,20220421185510,MKID004,8749,1200
147,31509,20220409115246,20220409115347,MKID004,19906,500
148,31837,20220427143436,20220427143618,MKID004,1640,300


In [37]:
# 결측치 없음
buy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   profile_id      150 non-null    int64 
 1   ss_id           150 non-null    int64 
 2   log_time        150 non-null    int64 
 3   act_target_dtl  150 non-null    object
 4   album_id        150 non-null    int64 
 5   payment         150 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 7.2+ KB


# 7. search

In [38]:
### 검색을 통한 시청 데이터 ###
# 어떻게 사용할지 추후 고민
search_df

,profile_id,ss_id,log_time,act_target_dtl,album_id
0,59,20220303183951,20220303183914,MKID017,2141
1,59,20220303183951,20220303183914,MKID017,2141
2,59,20220303183951,20220303183914,MKID017,2141
3,59,20220304181433,20220304181525,MKID017,512
4,59,20220304181433,20220304182447,MKID017,2141
...,...,...,...,...,...
6469,32965,20220423161110,20220423164503,MKID017,23083
6470,32965,20220424065420,20220424065456,MKID017,15080
6471,32965,20220425073255,20220425073358,MKID017,15080
6472,32965,20220427062749,20220427063353,MKID017,16497


In [39]:
# 결측치 없음
search_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6474 entries, 0 to 6473
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   profile_id      6474 non-null   int64 
 1   ss_id           6474 non-null   int64 
 2   log_time        6474 non-null   int64 
 3   act_target_dtl  6474 non-null   object
 4   album_id        6474 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 253.0+ KB


# 8. meta_plus

In [40]:
### 콘텐츠 확장 정보 ###
meta_plus_df

,album_id,keyword_type,keyword_name,keyword_value
0,749,05010101,시리즈,0
1,749,05010302,EBS,0
2,749,05010401,3세,0
3,749,05010402,4세,0
4,749,05010407,1세,0
...,...,...,...,...
767943,11629,05040104,신나는,0
767944,11629,0504010b,따라하고 싶은,0
767945,11629,05070e01,인체,0
767946,11629,05080401,캐릭터 친구들,0


In [41]:
# 결측치 없음
meta_plus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767948 entries, 0 to 767947
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   album_id       767948 non-null  int64 
 1   keyword_type   767948 non-null  object
 2   keyword_name   767948 non-null  object
 3   keyword_value  767948 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 23.4+ MB


In [42]:
meta_plus_df['keyword_name'].nunique()

1062

In [43]:
meta_plus_df['keyword_type'].nunique()

1101

In [44]:
# keyword_type이 keyword_name보다 많으므로 keyword_name으로 묶어봄
keywords = defaultdict(set)
for idx, row in meta_plus_df.iterrows():
    keywords[row['keyword_name']].add(row['keyword_type'])

In [45]:
# 같은 키워드인데 type이 다른 것이 존재
# 왜 같은 키워드인데 type이 다를까? -> 같은 키워드이면 같은 type으로 통일해도 될까?
for name, _type in keywords.items():
    if len(_type) != 1:
        print(name, _type)

사랑 {'05090702', '05030901'}
호기심 {'0507061f', '0509070a'}
봄 {'05030a03', '050b0204'}
교통기관 {'050b0202', '050c0407'}
초등학생 {'0508030f', '050a0131'}
운동 {'05090804', '05090905'}
성격 {'050c0e03', '050c0802'}
시간 {'050c0e1c', '050c0908'}
외모 {'050c0e02', '050c0803'}
동물 {'05080101', '050c0e0d', '050c0a01'}
색깔 {'050c0905', '050c0e09', '05030703'}
가족 {'050c0e06', '05030801'}
음악 {'050c0d02', '05030704'}
장난감 {'0508080b', '050c0e11'}
게임 {'050c0e17', '05030b02'}
여행 {'050c080a', '05030303', '050c0e28'}
취미 {'050c0e1b', '05030318'}
생활도구 {'050b0206', '050c0406'}
곤충 {'05080126', '050c0e0e'}
날씨 {'05030a07', '050c0e16'}
o {'050c1403', '050c1304'}
과일 {'05080705', '050c0e0b'}
미술 {'050c0d01', '05030701'}
식물 {'050c0a02', '05080117'}
모양 {'050c0906', '050c0e0f'}
음식 {'050c0e0a', '05080701'}
겨울 {'050b0201', '05030a06'}
눈 {'050a010c', '05030a0a'}
성교육 {'050a0123', '05070e0c'}
나라 {'050c0e24', '05070501'}
i {'050c1303', '050c1402'}
a {'050c1401', '050c1301'}
알레르기 {'050a010b', '05070e0a'}
우리나라 {'050b020a', '050c0408'}
쇼핑 {

## history와 watch_e data

In [46]:
# 말도 안되는 데이터가 다수 존재
# profile_id와 album_id가 같고 log_time의 날짜까지 같은데 시간은 얼마 차이가 나지 않는 데이터 존재
# 아래의 watch_e의 데이터를 보면 watch_time은 462초(7분 42초) 또는 463초인데 history의 log_time은 몇 초, 몇 분 간격으로 데이터가 다수 존재
# 아래의 데이터 중 가장 빠른 log_time인 20220427145520부터 시작하더라도 종료시간까지 같이 보면 한 번밖에 볼 수 없음
temp_h = history_df[history_df['profile_id'] == 33032]
temp_h = temp_h[temp_h['album_id'] == 381]
temp_h

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
1005623,33032,20220427145274,20220427145520,MKID003,381,NaN,N,N
1005624,33032,20220427145274,20220427145549,MKID003,381,NaN,Y,N
1005627,33032,20220427145841,20220427145826,MKID003,381,NaN,N,N
1005628,33032,20220427145841,20220427145867,MKID003,381,NaN,N,N
1005629,33032,20220427145902,20220427150141,MKID003,381,NaN,Y,N
1005630,33032,20220427145902,20220427150170,MKID003,381,NaN,N,N
1005632,33032,20220427145902,20220427150176,MKID003,381,NaN,Y,N
1005633,33032,20220427150248,20220427150289,MKID003,381,NaN,N,N
1005641,33032,20220427155091,20220427155559,MKID003,381,NaN,N,N
1005643,33032,20220427155091,20220427155601,MKID003,381,NaN,Y,N


In [47]:
temp_w = watch_e_df[watch_e_df['profile_id'] == 33032]
temp_w = temp_w[temp_w['album_id'] == 381]
temp_w

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
892753,33032,20220427145537,20220427145556,MKID049,381,462,463,1
892754,33032,20220427145274,20220427145592,MKID049,381,462,463,1
892758,33032,20220427145841,20220427145870,MKID049,381,463,464,1
892759,33032,20220427145938,20220427145956,MKID049,381,462,463,1
892760,33032,20220427150199,20220427150111,MKID049,381,462,463,1
892762,33032,20220427145902,20220427150153,MKID049,381,462,463,1
892763,33032,20220427145902,20220427150177,MKID049,381,463,464,1
892764,33032,20220427150324,20220427150350,MKID049,381,462,464,1
892784,33032,20220427155091,20220427155529,MKID049,381,462,463,1
892785,33032,20220427155091,20220427155601,MKID049,381,462,463,1


In [48]:
# history의 log_time을 년 월 일 시 까지만 저장하였음
# history에 직접하지 않고 temp_history에 한 이유: 마음대로 이런 기준을 가지고 처리해도 될까 싶어서...?
temp_history = history_df[:]
temp_history['log_time'] = temp_history['log_time']//10000
temp_history

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,2022030111,MKID003,15,NaN,Y,N
1,3,20220301115653,2022030111,MKID003,16,NaN,Y,N
2,3,20220301115653,2022030111,MKID003,17,NaN,Y,N
3,3,20220301115653,2022030112,MKID003,18,NaN,Y,N
4,3,20220301115653,2022030112,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,2022042715,MKID003,381,NaN,Y,N
1005647,33032,20220427155091,2022042715,MKID003,381,NaN,Y,N
1005648,33032,20220427155839,2022042715,MKID003,125,NaN,Y,N
1005649,33032,20220427155706,2022042715,MKID003,125,NaN,Y,N


In [49]:
temp_history.duplicated(['profile_id', 'log_time', 'album_id']).sum()

294040

In [50]:
# profile_id, log_time(년월일시), album_id가 같은 데이터 삭제
temp_history = temp_history.drop_duplicates(['profile_id', 'log_time', 'album_id'])

In [51]:
temp_history

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,2022030111,MKID003,15,NaN,Y,N
1,3,20220301115653,2022030111,MKID003,16,NaN,Y,N
2,3,20220301115653,2022030111,MKID003,17,NaN,Y,N
3,3,20220301115653,2022030112,MKID003,18,NaN,Y,N
4,3,20220301115653,2022030112,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005634,33032,20220427151499,2022042715,MKID003,7105,NaN,N,N
1005636,33032,20220427151499,2022042715,MKID003,2297,NaN,Y,N
1005637,33032,20220427151499,2022042715,MKID003,327,NaN,N,N
1005639,33032,20220427151499,2022042715,MKID003,1725,NaN,Y,N
